In [200]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.ops.logging_ops import Print


In [201]:
fp = "QB data.xlsx"

In [202]:
def prep(file_path):
    
    df_list = []
    xls = pd.ExcelFile(file_path)
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name, nrows=25)
        df['Year'] = sheet_name
        df_list.append(df)
    new = pd.concat(df_list, ignore_index=True)
    # print(new)
    return new
    


In [203]:
all = prep(fp)
all['Awards'] = all['Awards'].apply(lambda x: x[2] if isinstance(x, str) and len(x) >= 3 else "")
all['Year'] = all['Year'].astype(int)
# all.to_csv("aaa")
# print(all)

In [204]:
Train = all[all['Year'].apply(lambda x: str(x)[-1] not in ['0', '5'])]
Test = all[all['Year'].apply(lambda x: str(x)[-1] in ['0', '5'])]

X_train = Train.drop(columns=['Rk', 'Player', 'Team', 'Conf', 'Awards'], axis=1)
Y_train = Train['Rk']

X_test_name = Test['Player']
X_test_year = Test['Year']
X_test = Test.drop(columns=['Rk', 'Player', 'Team', 'Conf', 'Awards'], axis=1)
Y_test = Test['Rk']




In [205]:
print(X_test_name)

75           Mac Jones*
76          Kyle Trask*
77         Zach Wilson*
78          Sam Howell*
79      Dillon Gabriel*
             ...       
720          Mike Fouts
721         Patrick Nix
722    Mark Butterfield
723           Mike Groh
724       Tony Graziani
Name: Player, Length: 150, dtype: object


In [206]:
c= X_train.loc[1,'Year']
print(type(c))

<class 'numpy.int32'>


In [207]:
print(X_train, Y_train)

      G  Cmp  Att  Cmp%   Yds  TD   TD%  Int  Int%   Y/A   AY/A   Y/C    Y/G  \
0    15  363  555  65.4  4903  36   6.5   11   2.0   8.8   9.24  13.5  326.9   
1    14  364  470  77.4  4508  45   9.6    3   0.6   9.6  11.22  12.4  322.0   
2    14  302  417  72.4  3941  24   5.8    6   1.4   9.5   9.95  13.0  281.5   
3    13  318  476  66.8  3883  32   6.7    9   1.9   8.2   8.65  12.2  298.7   
4    12  236  327  72.2  3812  40  12.2    4   1.2  11.7  13.55  16.2  317.7   
..   ..  ...  ...   ...   ...  ..   ...  ...   ...   ...    ...   ...    ...   
770  11  196  356  55.1  2608  21   5.9   16   4.5   7.3   6.48  13.3  237.1   
771  11  213  388  54.9  2563  12   3.1   14   3.6   6.6   5.60  12.0  233.0   
772  11  171  280  61.1  2547  15   5.4   11   3.9   9.1   8.40  14.9  231.5   
773  11  179  373  48.0  2490  17   4.6   26   7.0   6.7   4.45  13.9  226.4   
774  11  177  304  58.2  2490  17   5.6    8   2.6   8.2   8.13  14.1  226.4   

      Rate  Year  
0    157.1  2023  
1

In [208]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [209]:
print(X_train.shape, Y_train.shape)

(625, 15) (625,)


In [210]:
input_weights = tf.constant([1.0, 1.0, 1.0, 1.0, 1.0, 2.0])

In [211]:
model = tf.keras.Sequential([
            
             # tf.keras.WeightedInputLayer(weights=input_weights, input_shape=(6,)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
model.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])

In [212]:
model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_data=(X_test, Y_test))

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0159 - loss: 193049.5156 - val_accuracy: 0.0400 - val_loss: 3892.3474
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0219 - loss: 7363.1274 - val_accuracy: 0.0400 - val_loss: 2835.6624
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0334 - loss: 3025.8997 - val_accuracy: 0.0333 - val_loss: 638.4032
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0388 - loss: 381.7687 - val_accuracy: 0.0267 - val_loss: 104.1273
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0359 - loss: 85.6739 - val_accuracy: 0.0267 - val_loss: 68.8661
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0226 - loss: 42.6181 - val_accuracy: 0.0267 - val_loss: 63.7052
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0209 - loss: 40.5209 - val_accuracy: 0.0333 - val_loss: 64.3087
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0262 - loss: 37.6988 - va

In [213]:
predictions = model.predict(X_test)
print(type(predictions.tolist()))
# print(predictions)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
<class 'list'>


In [214]:
print(type(X_test_name))

<class 'pandas.core.series.Series'>


In [215]:
new = pd.DataFrame()

new.reset_index(drop=True, inplace=True)
X_test_year.reset_index(drop=True, inplace=True)
X_test_name.reset_index(drop=True, inplace=True)

In [216]:
asa = pd.DataFrame(predictions)

new['Prediction'] = asa
new['Name'] = X_test_name
new['Year'] = X_test_year


In [217]:

print(new)

     Prediction              Name  Year
0     -0.433503        Mac Jones*  2020
1      3.997786       Kyle Trask*  2020
2     11.305869      Zach Wilson*  2020
3     11.854365       Sam Howell*  2020
4     15.129363   Dillon Gabriel*  2020
..          ...               ...   ...
145   15.932475        Mike Fouts  1995
146   16.760530       Patrick Nix  1995
147   16.984861  Mark Butterfield  1995
148   14.431510         Mike Groh  1995
149   19.254381     Tony Graziani  1995

[150 rows x 3 columns]


In [218]:
grouped = new.groupby('Year')
years = []

for year, group in grouped:
    globals()[f"df_{year}"] = group
    years.append(f"df_{year}")
    

In [219]:
for i in years:
    globals()[f"{i}"] = globals()[f"{i}"].sort_values(by='Prediction', ascending=True).head(3)

top = Test
top = top.iloc[0:0]
print(top)
top.to_csv("Top_QB.csv", index=False)

for i in years:
    for name in globals()[f"{i}"]['Name']:
        # print(name)
        r = Test[Test.apply(lambda row: row.astype(str).str.contains(name).any(), axis=1)] 
        r.reset_index(drop=True, inplace=True)
        # # print(r.head(1))
        
        r.to_csv("Top_QB.csv", mode='a', index=False, header=False)

   


Empty DataFrame
Columns: [Rk, Player, Team, Conf, G, Cmp, Att, Cmp%, Yds, TD, TD%, Int, Int%, Y/A, AY/A, Y/C, Y/G, Rate, Awards, Year]
Index: []
